## 데이터셋 준비

In [1]:
PATH_DEFECT = 'dataset/Defect_images/'
PATH_MASK = 'dataset/Mask_images/'
PATH_NODEFECT = 'dataset/NODefect_images/'

### Dataset 불러오기
[데이타셋 링크](https://www.aitex.es/afid)  


In [3]:
!pip install opencv-python

     |████████████████████████████████| 50.4 MB 23.2 MB/s eta 0:00:01


In [4]:
import cv2
import os
import glob
import shutil
import random
import string
import numpy as np

In [5]:
random.seed(0)
defect_list = glob.glob(PATH_DEFECT + '*.png')
mask_list = glob.glob(PATH_MASK + '*.png')
pass_list = glob.glob(PATH_NODEFECT + '**/*.png')

In [18]:
defect_list[0].split('/')[-1].split('_')[0]

'0061'

In [7]:
mask_list[0]

'dataset/Mask_images/0042_019_02_mask.png'

In [8]:
pass_list[0]

'dataset/NODefect_images/2306894-210033u/0004_000_02.png'

In [19]:
# Match defect-mask pairs
new_defect_list = list()
new_mask_list = list()
for defect in defect_list:
    num = defect.split('/')[-1].split('_')[0]
    for mask in mask_list:
        num_mask = mask.split('/')[-1].split('_')[0]
        if num == num_mask:
            new_defect_list.append(defect)
            new_mask_list.append(mask)
            break
defect_list = new_defect_list
mask_list = new_mask_list

## 첫 발송데이터 생성

In [20]:
# The first dataset given
if os.path.exists('dataset/1') is False:
    os.mkdir('dataset/1')
for file_name in pass_list + defect_list:
    if random.randint(0, 9) < 2:
        barcode = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
        shutil.copy2(file_name, 'dataset/1/' + barcode + '.png')

## 두번째 발송 데이터 생성

In [21]:
# The second dataset
if os.path.exists('dataset/2') is False:
    os.mkdir('dataset/2')
if os.path.exists('dataset/2/OK') is False:
    os.mkdir('dataset/2/OK')
if os.path.exists('dataset/2/FAIL') is False:
    os.mkdir('dataset/2/FAIL')
idx = 0
for file_name in pass_list:
    img = cv2.imread(file_name)
    height, width, _ = img.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height and random.randint(0, 9) < 2:
            patch = img[:, w:w+height, :]
            cv2.imwrite('dataset/2/OK/%04d.png' % idx, patch)
            idx += 1

patch_list = list()
for item in zip(defect_list, mask_list):
    defect, mask = item
    img_d = cv2.imread(defect)
    img_m = cv2.imread(mask)
    
    height, width, _ = img_d.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height:
            patch = img_d[:, w:w+height, :]
            patch_d = img_m[:, w:w+height, :]
            if patch_d.sum() > 0:
                patch_list.append(patch)
random.shuffle(patch_list)
patch_list_fraction = patch_list[:len(patch_list)//3]
for idx, patch in enumerate(patch_list_fraction):
    cv2.imwrite('dataset/2/FAIL/%04d.png' % idx, patch)

## 세번째 데이터 생성

In [22]:
# The third dataset
if os.path.exists('dataset/3') is False:
    os.mkdir('dataset/3')
if os.path.exists('dataset/3/OK') is False:
    os.mkdir('dataset/3/OK')
if os.path.exists('dataset/3/FAIL') is False:
    os.mkdir('dataset/3/FAIL')
if os.path.exists('dataset/3/MASK') is False:
    os.mkdir('dataset/3/MASK')
idx = 0
for file_name in pass_list:
    img = cv2.imread(file_name)
    height, width, _ = img.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height and random.randint(0, 9) < 3:
            patch = img[:, w:w+height, :]
            cv2.imwrite('dataset/3/OK/%04d.png' % idx, patch)
            idx += 1
patch_pair_list = list()
for item in zip(defect_list, mask_list):
    defect, mask = item
    img_d = cv2.imread(defect)
    img_m = cv2.imread(mask)
    height, width, _ = img_d.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height:
            patch = img_d[:, w:w+height, :]
            patch_d = img_m[:, w:w+height, :]
            if patch_d.sum() > 0:
                patch_pair_list.append((patch, patch_d))
random.shuffle(patch_pair_list)
for idx, pair in enumerate(patch_pair_list):
    pathc, patch_d = pair
    cv2.imwrite('dataset/3/FAIL/%04d.png' % idx, patch)
    cv2.imwrite('dataset/3/MASK/%04d.png' % idx, patch_d)

## 실전 데이터 생성

In [23]:
if os.path.exists('dataset/data/') is False:
    os.mkdir('dataset/data/')
if os.path.exists('datasets/tfrecodrds/') is False:
    os.mkdir('dataset/tfrecords/')
if os.path.exists('dataset/model/') is False:
    os.mkdir('dataset/model/')
if os.path.exists('dataset/input_data') is False:
    os.mkdir('dataset/input_data')
if os.path.exists('dataset/output_csv') is False:
    os.mkdir('dataset/output_csv')
    
idx = 0
for file_name in pass_list:
    img = cv2.imread(file_name)
    height, width, _ = img.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height and random.randint(0, 9) < 5:
            patch = img[:, w:w+height, :]
            cv2.imwrite('dataset/input_data/ok_%04d.png' % idx, patch)
            idx += 1
patch_pair_list = list()
for item in zip(defect_list, mask_list):
    defect, mask = item
    img_d = cv2.imread(defect)
    img_m = cv2.imread(mask)
    height, width, _ = img_d.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height:
            patch = img_d[:, w:w+height, :]
            patch_d = img_m[:, w:w+height, :]
            if patch_d.sum() > 0:
                patch_pair_list.append((patch, patch_d))
random.shuffle(patch_pair_list)
for idx, pair in enumerate(patch_pair_list):
    pathc, patch_d = pair
    cv2.imwrite('dataset/input_data/fail_%04d.png' % idx, patch)